## Part 2
#### Load training file and create dictionary containing the occurrences of each annotation in the training set

In [1]:
def load_original_train():
    with open("train", encoding="utf-8") as file:
        train_list = file.readlines()
        train_list = [x.strip() for x in train_list]
        return train_list
    
train_list = load_original_train()

In [2]:
annotationDict = {"O": 0, "I-positive": 0, "B-positive": 0, "I-negative":0, "B-negative": 0, "I-neutral":0, "B-neutral":0}

for i in range(len(train_list)):
    annotated_word = train_list[i]
    each_word = annotated_word.split(" ")
    if (len(each_word) == 2):
        if each_word[1] == "O":
            annotationDict["O"] += 1
        elif each_word[1] == "I-positive":
            annotationDict["I-positive"] += 1
        elif each_word[1] == "B-positive":
            annotationDict["B-positive"] += 1
        elif each_word[1] == "I-negative":
            annotationDict["I-negative"] += 1
        elif each_word[1] == "B-negative":
            annotationDict["B-negative"] += 1
        elif each_word[1] == "I-neutral":
            annotationDict["I-neutral"] += 1
        elif each_word[1] == "B-neutral":
            annotationDict["B-neutral"] += 1

print(annotationDict)

{'O': 91753, 'I-positive': 1653, 'B-positive': 2613, 'I-negative': 443, 'B-negative': 1299, 'I-neutral': 5272, 'B-neutral': 5722}


#### (a) Write a function that estimates the emission parameters from the training set using MLE

In [3]:
def estimate_emission(word, tag):
    countTop = 0
    countBottom = annotationDict[tag]
    for i in range(len(train_list)):
        annotated_word = train_list[i]
        each_word = annotated_word.split(" ")
        if len(each_word) == 2:
            if (each_word[0] == word and each_word[1] == tag):
                countTop += 1

#         print(countTop)
#         print(countBottom)
    return countTop/countBottom

estimate_emission("service", "O")

0.0001198870881606051

#### (b) Replace the words that appear less than k times in the training set with a special token #UNK# before training. This leads to a “modified training set”.

In [4]:
import datetime

def modify_trainingset(k):
        wordCountDict = {}
        for i in range(len(train_list)):
            annotated_word = train_list[i]
            each_word = annotated_word.split(" ")
            if each_word[0] in wordCountDict:
                wordCountDict[each_word[0]] += 1
            else:
                wordCountDict[each_word[0]] = 1
        
       
        wordToBeReplacedDict = {}
        wordToBeReplacedList = []
        for key in wordCountDict:
            if wordCountDict[key] < k:
                wordToBeReplacedList.append(key)

        start = datetime.datetime.now()
        for i in range(len(train_list)): 
            annotated_word = train_list[i]
            each_word = annotated_word.split(" ")
            if each_word[0] in wordToBeReplacedList:
                train_list[i] = "#UNK# " + each_word[1]


        modifiedTotalTrainSet = "\n".join(train_list)

        f = open("modified_train","w+")
        f.write(modifiedTotalTrainSet)
                            
        end = datetime.datetime.now()
        elapsed = end - start
        print(elapsed)

modify_trainingset(3)

0:00:35.356392


#### Loads the modified training set into a list, and count the occurences of each word in the list and store it into a dictionary called modifiedWordDict

In [5]:
def load_modified_train_file():
    with open("modified_train", encoding="utf-8") as file:
        modified_train_list = file.readlines()
        modified_train_list = [x.strip() for x in modified_train_list]
        return modified_train_list
        
modified_train_list = load_modified_train_file()

In [6]:
def get_word_dict():
    modifiedWordDict = {}        
    for i in range(len(modified_train_list)):
        annotated_word = modified_train_list[i]
        each_word = annotated_word.split(" ")
        if each_word[0] in modifiedWordDict:
            modifiedWordDict[each_word[0]] += 1
        else:
            modifiedWordDict[each_word[0]] = 1

    return modifiedWordDict
    
modifiedWordDict = get_word_dict()

In [7]:
def count_word_tag_pair_dict():
    word_tag_pair_dict= {}
    for word_tag_pair in modified_train_list:
        if word_tag_pair_dict.get(word_tag_pair) == None:
            word_tag_pair_dict[word_tag_pair] = 1
        else:
            word_tag_pair_dict[word_tag_pair] += 1
    return word_tag_pair_dict

    
wordTagPairDict = count_word_tag_pair_dict()

#### Implement a fixed emission function

In [8]:
def estimate_emission_fix(word, tag):
        countTop = 0
        countBottom = annotationDict[tag]
        if word in modifiedWordDict:
            # do nothing
            word = word
            
        else:
            word = "#UNK#"

        for i in range(len(modified_train_list)):
            annotated_word = modified_train_list[i]
            each_word = annotated_word.split(" ")
            if (each_word[0] == word and each_word[1] == tag):
                countTop += 1
        return countTop/countBottom

estimate_emission_fix("I", "O")

0.011203993329918368

#### Loop through the words once and store the emission value of each word tag pair into a giant dictionary called giantEmissionDict

In [16]:
def store_estimate_emission_fix():
    giantEmissionDict = {}
    for word_tag_pair in modified_train_list:
        if word_tag_pair != "":
            word_tag_pair_list = word_tag_pair.split(" ")
            countTop = wordTagPairDict[word_tag_pair]
            countBottom = annotationDict[word_tag_pair_list[len(word_tag_pair_list)-1]]
            emission = countTop/countBottom
            if giantEmissionDict.get(word_tag_pair) == None:
                giantEmissionDict[word_tag_pair] = countTop/countBottom
    return giantEmissionDict
    

giantEmissionDict = store_estimate_emission_fix()
# print(modified_train_list)
# print(giantEmissionDict)

#### (c) Implement a simple sentiment analysis system that produces the tag

In [17]:
def sentiment_analysis(word):
    all_tags = ["O", "B-positive", "I-positive", "B-negative", "I-negative", "B-neutral", "I-neutral"]
    scoreDict = {}
    # if word never appeared before, let it be #UNK#
    if word in modifiedWordDict:
        word = word
    else:
        word = "#UNK#"
    for tag in all_tags:
        possible_word_tag = word + " " + tag
        if giantEmissionDict.get(possible_word_tag) == None:
            scoreDict[tag] = 0
        else:
            scoreDict[tag] = giantEmissionDict[possible_word_tag]
            
    maximumScore = max(scoreDict.values())
    return max(scoreDict, key=scoreDict.get)

   
sentiment_analysis("ambience")

'B-positive'

#### Loads dev.in file and store as dev_in_list

In [18]:
def load_files():
    with open("dev.in", encoding="utf-8") as file:
        dev_in_list = file.readlines()
        dev_in_list = [x.strip() for x in dev_in_list]
        return dev_in_list
        
dev_in_list = load_files()

In [19]:
import datetime
def predict_sentiment():
    wordDict = {}
    f = open("dev.p2.out","w+")
    giantStringToBeWritten = ""
    start = datetime.datetime.now()
    for i in range(len(dev_in_list)):
        each_word = dev_in_list[i]
        if each_word in wordDict:
            tag = wordDict[each_word]
            giantStringToBeWritten += each_word +  " " + tag + "\n"

        else:
            if each_word != "":
                tag = sentiment_analysis(each_word)
                wordDict[each_word] = tag 
                giantStringToBeWritten += each_word +  " " + tag + "\n"

            else:
                giantStringToBeWritten += " \n"


    f.write(giantStringToBeWritten)
    end = datetime.datetime.now()
    elapsed = end - start
    print(elapsed)

predict_sentiment()

0:00:00.098191


### Part 3 
#### Function to estimate Transition Parameters using MLE

#### Creating a giant_each_tweet_tag that contains only the states of each sentence

In [30]:
def separate_tweets():
    countStart = 1
    # counting starts
    giant_each_tweet = []
    each_tweet = []
    totalStateNumber = {}
    for i in range(len(modified_train_list)):
        annotated_word = modified_train_list[i]
        each_word = annotated_word.split(" ")
        if each_word == ['']:
            countStart += 1
            giant_each_tweet.append(each_tweet)
            each_tweet = []

        else:
            each_tweet.append(each_word[len(each_word)-1])
    for i in range(len(giant_each_tweet)):
        giant_each_tweet[i].insert(0, 'START')
        giant_each_tweet[i].append('STOP')
    
    for each_tweet in giant_each_tweet:
        for tag in each_tweet:
            if totalStateNumber.get(tag) == None:
                totalStateNumber[tag] = 1
            else:
                totalStateNumber[tag] += 1

    return giant_each_tweet, countStart, totalStateNumber
giantTaggedEachTweet, countStart, totalStateNumber = separate_tweets()
print(totalStateNumber)
# print(giantEmissionDict)

{'START': 7093, 'O': 91733, 'B-neutral': 5720, 'STOP': 7093, 'B-positive': 2613, 'I-positive': 1653, 'I-neutral': 5268, 'B-negative': 1299, 'I-negative': 443}


In [31]:
def store_estimate_transition():
    giantTransitionDict = {}
    finalGiantTransitionDict = {}
    for taggedEachTweet in giantTaggedEachTweet:
        for i in range(len(taggedEachTweet) - 1):
            if giantTransitionDict.get(taggedEachTweet[i]+" "+taggedEachTweet[i+1]) == None:
                giantTransitionDict[taggedEachTweet[i]+" "+taggedEachTweet[i+1]] = 1
            else:
                giantTransitionDict[taggedEachTweet[i]+" "+taggedEachTweet[i+1]] += 1
    for eachTransition in giantTransitionDict:
        eachTransitionList = eachTransition.split(" ")
        startState = eachTransitionList[0]
        countTop = giantTransitionDict[eachTransition]
        countBottom = totalStateNumber[startState]
        transitionParam = countTop/countBottom
        finalGiantTransitionDict[eachTransition] = transitionParam
        
    return finalGiantTransitionDict
    
giantTransitionDict = store_estimate_transition()
print(giantTransitionDict)

{'START O': 0.8732553221485972, 'O O': 0.8316963361058725, 'O B-neutral': 0.05734032463780755, 'B-neutral O': 0.5085664335664336, 'O STOP': 0.07398646070661594, 'O B-positive': 0.024996457109219147, 'B-positive O': 0.5667814772292384, 'START B-neutral': 0.05681657972649091, 'B-positive I-positive': 0.4041331802525832, 'I-positive O': 0.6091954022988506, 'B-neutral I-neutral': 0.46695804195804197, 'I-neutral I-neutral': 0.49297646165527714, 'I-neutral STOP': 0.0115793470007593, 'I-positive I-positive': 0.36116152450090744, 'O B-negative': 0.011980421440484886, 'B-negative O': 0.7028483448806775, 'START B-negative': 0.02735090934724376, 'I-neutral O': 0.4922171602126044, 'B-positive STOP': 0.023727516264829697, 'START B-positive': 0.042577188777668126, 'B-negative I-negative': 0.27174749807544263, 'I-negative O': 0.7720090293453724, 'I-positive STOP': 0.02601330913490623, 'B-neutral B-neutral': 0.005594405594405594, 'I-negative I-negative': 0.20316027088036118, 'B-neutral STOP': 0.017832

In [32]:
def split_into_sentences(dev_in_list):
    sentences = []
    each_sentence = []
    for word in dev_in_list:
        if word == "":
            sentences.append(each_sentence)
            each_sentence = []
        else:
            each_sentence.append(word)
        
    return sentences
sentences = split_into_sentences(dev_in_list)

## Viterbi

In [33]:
def find_path_backward(piList, sentence):
    path = []
    all_tags=["O","B-negative","B-positive","B-neutral","I-negative","I-positive","I-neutral"]
    if len(piList) == len(sentence):
#         print('sentence length and pilist length are same, move on')
        for i in range(len(piList)):
            # select the max of the last layer, use the max to find the previous value
            if i == 0: #its the start, i = 0 comes here
                lastLayer = piList[len(piList) - (i+1)]
                maxPiTag = max(lastLayer, key=lastLayer.get) 
                path.append('end')
            elif i == 1:
                path.append(maxPiTag)
            else: #i>0 comes here
                currentLayer = piList[len(piList) - (i+1)] #2nd last layer onwards
                targetedTag = path[i-1]
                scoreDict = {}
                for tag in currentLayer: 
                    if giantTransitionDict.get(tag + " " + targetedTag) != None:
                        scoreOfTag = giantTransitionDict[tag + " " + targetedTag] * currentLayer[tag]
                        scoreDict[tag] = scoreOfTag
                    
                bestScoreTag = max(scoreDict, key=scoreDict.get)
                path.append(bestScoreTag)
    singleStringToBeWritten = ""
    for j in range(len(sentence)):
        if sentence[j] == "":
            singleStringToBeWritten += "\n"
        else:
            singleStringToBeWritten += sentence[j] + " " + path[len(path)-(j+1)] + "\n"
    return singleStringToBeWritten

# expected: O, B-pos, I-pos, I-pos, I-pos, O, O, O
piList4 = [{'O': 0.10952242946302841, 'B-negative': 0.0019577571933592874, 'B-positive': 0.010624504726325918, 'B-neutral': 0.0009040105193951349}, {'O': 3.108819999877843e-05}, {'B-negative': 1.212411803650987e-09, 'B-positive': 3.5875587995634343e-09}, {'B-positive': 4.751733509355542e-11}]
sentence2 = ['AVOID', 'THE', 'PLACE', '']
find_path_backward(piList4, sentence2)

'AVOID O\nTHE O\nPLACE B-positive\n\n'

In [34]:
def viterbi(sentence):
    all_tags=["O","B-negative","B-positive","B-neutral","I-negative","I-positive","I-neutral"]
    sentence.append("") #so we can detect the end
    beginning = True
    piList = []
#     for word in sentence:
    for i in range(len(sentence)):
        word = sentence[i]
        if word in modifiedWordDict:
            word = word
        else:
            word = "#UNK#"
        
        piLayer = {}
        if beginning: #base case
            for tag in all_tags:
                if giantTransitionDict.get("START" + " " + tag) != None and giantEmissionDict.get(word + " " + tag) != None:
                    pi = giantTransitionDict["START" + " " + tag]*giantEmissionDict[word + " " + tag]
                    piLayer[tag] = pi
            piList.append(piLayer)
            beginning = False
        else: #recursive
            previousLayer = piList[len(piList) - 1] #last entry of piList
            if word == "": #end of sentence
                tempLayer = {} 
                for previous_tag in previousLayer:
                    if giantTransitionDict.get(previous_tag + " " + "STOP") != None:
                        pi = previousLayer[previous_tag]*giantTransitionDict[previous_tag + " " + "STOP"] #new score
                        tempLayer[previous_tag] = pi
                if len(tempLayer) != 0:
                    maxPiTag = max(tempLayer, key=tempLayer.get)
                    maximumPiScore = max(tempLayer.values())
                    lastLayer = {}
                    lastLayer[maxPiTag] = maximumPiScore
                    piList.append(lastLayer)
                    singleStringToBeWritten = find_path_backward(piList, sentence)
                    return singleStringToBeWritten
                else:
                    return "ERROR"

            else: #if not end of sentence
                for tag in all_tags:
                    tempLayer = {}
                    for previous_tag in previousLayer:
                        if giantTransitionDict.get(previous_tag + " " + tag) != None and giantEmissionDict.get(word + " " + tag) != None:
                            pi = previousLayer[previous_tag]*giantTransitionDict[previous_tag + " " + tag]*giantEmissionDict[word + " " + tag]
                            tempLayer[previous_tag] = pi
                    # what if the length is 0? it will never transit to the tag
                    if len(tempLayer) != 0:
                        # pick the max here
                        maxPiTag = max(tempLayer, key=tempLayer.get) 
                        maxPiScore = max(tempLayer.values())
                        piLayer[tag] = maxPiScore
                piList.append(piLayer)


giantStringToBeWritten = ""
for sentence in sentences:
# singleStringToBeWritten = viterbi(['AVOID', 'THE', 'PLACE', ''])
    singleStringToBeWritten = viterbi(sentence)
    giantStringToBeWritten += singleStringToBeWritten
print(giantStringToBeWritten)
f = open("dev.p3.out","w+")
f.write(giantStringToBeWritten)


Happy O
week O
( O
end O
) O
( O
with O
Jumna O
at O
Diamond B-neutral
City I-neutral
( I-neutral
DC I-neutral
) I-neutral
Mall I-neutral
) O
— O
https://t.co/KeHKnPNjoq O

my O
mum O
mixed O
matcha O
with O
latte O
for O
breakfast O
: O
" O
) O
what O
a O
great O
start O

Kinda O
sad O
that O
anytime O
we O
talk O
about O
#Ghostbusters O
now O
is O
to O
make O
fun O
and O
laugh O
at O
the O
movie O
and O
the O
ridiculous O
shit O
in O
it O
#NomoreSJWremakes O

The O
Friday O
#vinyl O
club O
is O
spinning O
the O
classic O
Fleetwood O
Mac O
album O
Rumours O
tmrw O
. O

@helsbels1181 O
Thankyou O
very O
much O
Helen O
, O
Everyones O
kind O
words O
and O
prayers O
are O
really O
appreciated O
x O

I O
feel O
like O
the O
underrated O
actors O
on O
#chicagopd O
make O
the O
show O
. O
Amy B-neutral
, I-neutral
LaRoyce I-neutral
, I-neutral
Burgess O
... O

@elllleeeenn O
and O
im O
gonna O
shine O
homie O
until O
my O
heart O
stop O

Tcu O
Lowkey O
IS O
my O
dream O
college O
.. O
just 

139410

## Viterbi try 3